In [1]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers

In [2]:
users_df = pd.read_csv('../data/users.csv')
books_df = pd.read_csv('../data/books.csv')
ratings_df = pd.read_csv('../data/ratings.csv')

In [3]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206447 entries, 0 to 206446
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   uid       206447 non-null  int64  
 1   location  206447 non-null  object 
 2   age       122881 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.7+ MB


In [4]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134692 entries, 0 to 134691
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 134692 non-null  object
 1   book_title           134692 non-null  object
 2   book_author          134692 non-null  object
 3   year_of_publication  134692 non-null  int64 
 4   publisher            134692 non-null  object
 5   summary              134692 non-null  object
 6   category             134692 non-null  object
dtypes: int64(1), object(6)
memory usage: 7.2+ MB


In [5]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031175 entries, 0 to 1031174
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   uid     1031175 non-null  int64 
 1   isbn    1031175 non-null  object
 2   rating  1031175 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 23.6+ MB


In [6]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

TypeError: NodeConfig.__init__() missing 1 required positional argument: 'scheme'

In [ ]:
books_df['summary'] = books_df['summary'].str.replace("\n", " ").str.strip()
books_df['category'] = books_df['category'].apply(lambda x: eval(x)[0] if isinstance(x, str) else x)

records = books_df.to_dict(orient='records')

# Helper function to index data
def index_data(records):
    for record in records:
        yield {
            "_index": "books",  # Name of the index
            "_source": record
        }

# Bulk index the data
helpers.bulk(es, index_data(records))